In [6]:
# 导入需要的包
import random
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
import base64


In [19]:
# 加密专用
class Encrypter:

    def __init__(self):
        self.sequence = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
        self.aes_iv = '0102030405060708'
        self.aes_mode = AES.MODE_CBC
        self.aes_key = '0CoJUm6Qyw8W8jud'
        self.rsa_public_key = '010001'
        self.rsa_modulus = '00e0b509f6259df8642dbc35662901'\
                    '477df22677ec152b5ff68ace615bb7b7'\
                    '25152b3ab17a876aea8a5aa76d2e417629'\
                    'ec4ee341f56135fccf695280104e0312e'\
                    'cbda92557c93870114af6c9d05c4f7f0c'\
                    '3685b7a46bee255932575cce10b424d813c'\
                    'fe4875d3e82047b97ddef52741d546b8e289dc6935b3ece0462db0a22b8e7'

    def random_ase_key(self):
        random_key = random.sample(self.sequence,16)
        random_key = ''.join(random_key)
        return random_key
    
    def aes_encrypt(self,plain_text,mode=AES.MODE_CBC,iv=None,key=None):
        # data padding
        # data_to_padd : need a byte objectencode return a bytes object,
        # The block boundary to use for padding
        plain_text = pad(data_to_pad=plain_text.encode(),block_size= AES.block_size)
        # create a encrypt object
        aes = AES.new(key=key.encode(),mode=mode,iv=iv.encode())
        # get the encrypted data
        encrypted_data = aes.encrypt(plaintext=plain_text)
        # use base64 to transfer the encrypted data
        # why ?
        result = base64.b64encode(encrypted_data).decode()
        return result

    def rsa_encrypt(self,data, public_key, modulus):
        # 部分情况下的运算会导致出现位数不足256位，需要进行填充
        num = pow(int(data[::-1].encode().hex(), 16), int(public_key, 16), int(modulus, 16))
        result = format(num, 'x').zfill(256)
        return result
    def aes_decrypt(self,encode_text,mode=AES.MODE_CBC,iv=None,key=None):
        # use base64 decode the data
        encode_data = base64.b64decode(encode_text.encode())
        # 
        aes = AES.new(key=key.encode(),mode=mode,iv=iv.encode())
        decrypted_data = aes.decrypt(encode_data)
        decrypted_data = unpad(decrypted_data,AES.block_size)
        return decrypted_data.decode()
    
    def get_encrypt(self,plain_text):
        random_key = self.random_ase_key()
        encode_data = self.aes_encrypt(plain_text=plain_text,mode=self.aes_mode,iv=self.aes_iv,key=self.aes_key)
        # 加密后再使用random_key加密
        encText = self.aes_encrypt(plain_text=encode_data,mode=self.aes_mode,iv=self.aes_iv,key=random_key)
        # 使用公钥对 random_key 进行加密，加密后传给服务器
        encKey = self.rsa_encrypt(random_key,self.rsa_public_key,self.rsa_modulus)

        form_data = {
            'encSecKey':encKey, #为何有时候是255位？计算问题...自动填充0即可
            'params':encText
            }
        return form_data

In [37]:
t = Encrypter()
form_data = t.get_encrypt('{"s":"张国荣","limit":"8","csrf_token":""}')

In [31]:
print(type('ss'.encode()))
# print('你'.encode())
# len('你好'.encode()) # encode返回byte形式,默认是utf-8
# print('你好'.encode(encoding='GBK'))
# len('你好'.encode(encoding='GBK'))
t = '你好'.encode(encoding='GBK')
print(t)
# print(t.decode(encoding='utf-8')) error
print(t.decode(encoding='GBK'))

<class 'bytes'>
b'\xc4\xe3\xba\xc3'
你好


In [40]:
format(11111,'x')

'2b67'

In [38]:
print(form_data)

{'encSecKey': '1749de18f0b8aa1da92527b1634ea798143e69aae67c5a9d3eff0d8f16c607e492812fa62e8e70838f7ab2ee7f24f0dcb7380c92223d6d5a8cdf4e2cd63f33e50d21f40790965340904ba86a17736573af70eba3cf7cfc3430315072558bead0c6ececa5b3c779aeb0f49e43bdddf0bdb1410da0446dd2aa5502223df7750be0', 'params': 'xUrka9EVQXyrvQTYzHyD+tYsuuG03D6Oj32pTjLF8VeUzjBibgCdyQnBHcWil+8EdpwiJ7LmGTBFEvoLgags85We6/ftiqU1yzJV37uf30U='}


In [39]:
import requests
HTTP_HEADER ={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                                      'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

def get_text_post(url,data):
    html = requests.post(url=url,data = data,headers=HTTP_HEADER)

    return html.text

text = get_text_post('https://music.163.com/weapi/search/suggest/web?csrf_token=',form_data)
text

'{"result":{"albums":[{"id":18937,"name":"I Am What I Am","artist":{"id":6457,"name":"张国荣","picUrl":"https://p2.music.126.net/7wDg-DUN511nwSSPmOU5Xw==/109951165745388126.jpg","alias":["Leslie Cheung","哥哥"],"albumSize":94,"picId":109951165745388126,"img1v1Url":"https://p2.music.126.net/6y-UleORITEDbvrOLV0Q8A==/5639395138885805.jpg","img1v1":0,"alia":["Leslie Cheung","哥哥"],"trans":null},"publishTime":1269273600000,"size":35,"copyrightId":7003,"status":1,"picId":109951166225429773,"mark":0},{"id":19057,"name":"Ultimate ","artist":{"id":6457,"name":"张国荣","picUrl":"https://p2.music.126.net/7wDg-DUN511nwSSPmOU5Xw==/109951165745388126.jpg","alias":["Leslie Cheung","哥哥"],"albumSize":94,"picId":109951165745388126,"img1v1Url":"https://p2.music.126.net/6y-UleORITEDbvrOLV0Q8A==/5639395138885805.jpg","img1v1":0,"alia":["Leslie Cheung","哥哥"],"trans":null},"publishTime":691516800000,"size":30,"copyrightId":7003,"status":3,"picId":102254581395221,"mark":0}],"artists":[{"id":6457,"name":"张国荣","picUrl":

In [10]:
import re
text = re.sub(r'null|false','None',text)
text = eval(text)

In [11]:
text['result'].keys()

dict_keys(['albums', 'artists', 'songs', 'playlists', 'order'])

In [15]:
for item in text['result']['artists']:
    print(item['id'])
    print(item['name'])

6457
张国荣


In [18]:
for item in text['result']['songs']:
    # print(item)
    print(item['id'])
    print(item['name'])

186436
玻璃之情
188204
沉默是金
186453
春夏秋冬
26620756
当爱已成往事


In [12]:
def handle_albums(albums):
    result = {}
    for i in albums:
        result[i['id']] = result[i['name']]
    return result

In [13]:
for i in  text['result']['albums']:
    print(i['id'])
    print(i['name'])

18937
I Am What I Am
19057
Ultimate 


In [ ]:
# 1、定义一个类，用来处理下载音乐，对外提供三个方法
# 2、一个方法是模糊查询，模糊查询返回['albums', 'artists', 'songs', 'playlists', 'order']信息，每个都是字典
# 3、一个方方法通过